# RAMP on predicting cyclist traffic in Paris

Authors: *Roman Yurchak (Symerio)*; also partially inspired by the air_passengers starting kit.


## Introduction

The dataset was collected with cyclist counters installed by Paris city council in multiple locations. It contains hourly information about cyclist traffic, as well as the following features,
 - counter name
 - counter site name
 - date
 - counter installation date
 - latitude and longitude
 
Available features are quite scarce. However, **we can also use any external data that can help us to predict the target variable.** 

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid

import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Loading the data with pandas

First, download the data files,
 - [train.parquet](https://github.com/rth/bike_counters/releases/download/v0.1.0/train.parquet)
 - [test.parquet](https://github.com/rth/bike_counters/releases/download/v0.1.0/test.parquet)

and put them to into the data folder.


Data is stored in [Parquet format](https://parquet.apache.org/), an efficient columnar data format. We can load the train set with pandas,

In [2]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [3]:
# Categorical and numerical ['date','pmer','tend','cod_tend','ff','t','u','vv','ww','w1','w2','nbas','pres','tend24']
# Just numerical [ ]

In [4]:
data_weather = pd.read_csv('submissions/external_data/external_data.csv')
data_weather = data_weather.loc[:, data_weather.columns.intersection(['date','pmer','ff','t','u','vv',
                                                                     'pres'])]

In [5]:
data_weather["date"] = pd.to_datetime(data_weather["date"], format = "%Y-%m-%d %H:%M:%S")

In [6]:
mask1 = (data.date >= "2020-09-01") & (data.date < "2020-09-02") & (data.site_name == '28 boulevard Diderot')

In [7]:
mask2 = (data_weather.date >= "2020-09-01") & (data_weather.date < "2020-09-02") 

In [8]:
data[mask1].shape

(46, 12)

In [9]:
data_weather[mask2]

,date,pmer,ff,t,u,vv,pres
3082,2020-09-01 00:00:00,102050,1.6,285.75,81,30000,100960
3083,2020-09-01 03:00:00,101990,1.1,283.95,88,25000,100900
3084,2020-09-01 06:00:00,102000,1.8,284.25,91,25000,100910
3085,2020-09-01 09:00:00,101970,2.9,291.25,60,19830,100910
3086,2020-09-01 12:00:00,101850,2.6,293.95,44,21000,100800
3087,2020-09-01 15:00:00,101740,4.0,293.65,41,30000,100690
3088,2020-09-01 18:00:00,101760,3.0,292.15,47,30000,100700
3089,2020-09-01 21:00:00,101880,2.7,289.35,53,30000,100810


In [10]:
data.head()

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585


# Here we face the first problem: the weather data is not hourly

In [11]:
data_test = pd.read_parquet(Path("data") / "test.parquet")

In [12]:
data.date.min()

Timestamp('2020-09-01 01:00:00')

In [13]:
data_test.date.max()

Timestamp('2021-09-09 23:00:00')

We can check general information about different columns,

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 455163 entries, 48321 to 928462
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype         
---  ------                     --------------   -----         
 0   counter_id                 455163 non-null  category      
 1   counter_name               455163 non-null  category      
 2   site_id                    455163 non-null  int64         
 3   site_name                  455163 non-null  category      
 4   bike_count                 455163 non-null  float64       
 5   date                       455163 non-null  datetime64[ns]
 6   counter_installation_date  455163 non-null  datetime64[ns]
 7   coordinates                455163 non-null  category      
 8   counter_technical_id       455163 non-null  category      
 9   latitude                   455163 non-null  float64       
 10  longitude                  455163 non-null  float64       
 11  log_bike_count             455163 non-null  floa

and in particular the number of unique entries in each column,

In [15]:
data.nunique(axis=0)

counter_id                     56
counter_name                   56
site_id                        30
site_name                      30
bike_count                    977
date                         8230
counter_installation_date      22
coordinates                    30
counter_technical_id           30
latitude                       30
longitude                      30
log_bike_count                977
dtype: int64

# NOW THE WEATHER

## We select a couple numerical features for now

In [16]:
data_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3322 entries, 0 to 3321
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    3322 non-null   datetime64[ns]
 1   pmer    3322 non-null   int64         
 2   ff      3322 non-null   float64       
 3   t       3322 non-null   float64       
 4   u       3322 non-null   int64         
 5   vv      3322 non-null   int64         
 6   pres    3322 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(4)
memory usage: 181.8 KB


In [17]:
data_weather.nunique(axis=0)

date    3321
pmer     465
ff       109
t        345
u         77
vv      1230
pres     459
dtype: int64

In [18]:
data_weather.isna().sum()

date    0
pmer    0
ff      0
t       0
u       0
vv      0
pres    0
dtype: int64

In [19]:
data_weather.dropna(inplace = True)

In [20]:
data

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count
48321,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48324,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147
48327,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000
48330,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438
48333,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585
...,...,...,...,...,...,...,...,...,...,...,...,...
928450,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,51.0,2021-08-08 18:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,3.951244
928453,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,1.0,2021-08-09 02:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,0.693147
928456,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,61.0,2021-08-09 08:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,4.127134
928459,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,44.0,2021-08-09 10:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,3.806662


In [21]:
data_weather

,date,pmer,ff,t,u,vv,pres
0,2021-01-01 00:00:00,100810,1.8,272.75,96,990,99680
1,2021-01-01 03:00:00,100920,1.7,271.25,98,210,99790
2,2021-01-01 06:00:00,100950,2.6,271.95,98,3660,99820
3,2021-01-01 09:00:00,101100,1.7,272.45,97,3500,99970
4,2021-01-01 12:00:00,101110,1.0,276.95,82,8000,100000
...,...,...,...,...,...,...,...
3317,2020-09-30 09:00:00,101540,4.4,289.95,82,18000,100480
3318,2020-09-30 12:00:00,101320,4.9,292.05,66,25000,100270
3319,2020-09-30 15:00:00,101140,4.1,291.55,72,25000,100090
3320,2020-09-30 18:00:00,101020,2.7,290.15,73,40820,99960


# Left merged

In [22]:
merged_train = pd.merge(data, data_weather, how='left')

In [23]:
merged_train = merged_train.replace(np.nan, 0)
merged_train

,counter_id,counter_name,site_id,site_name,bike_count,date,counter_installation_date,coordinates,counter_technical_id,latitude,longitude,log_bike_count,pmer,ff,t,u,vv,pres
0,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 02:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
1,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,1.0,2020-09-01 03:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.693147,101990.0,1.1,283.95,88.0,25000.0,100900.0
2,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,0.0,2020-09-01 04:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,0.000000,0.0,0.0,0.00,0.0,0.0,0.0
3,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,4.0,2020-09-01 15:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,1.609438,101740.0,4.0,293.65,41.0,30000.0,100690.0
4,100007049-102007049,28 boulevard Diderot E-O,100007049,28 boulevard Diderot,9.0,2020-09-01 18:00:00,2013-01-18,"48.846028,2.375429",Y2H15027244,48.846028,2.375429,2.302585,101760.0,3.0,292.15,47.0,30000.0,100700.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455212,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,51.0,2021-08-08 18:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,3.951244,101440.0,4.1,293.05,53.0,25000.0,100390.0
455213,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,1.0,2021-08-09 02:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,0.693147,0.0,0.0,0.00,0.0,0.0,0.0
455214,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,61.0,2021-08-09 08:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,4.127134,0.0,0.0,0.00,0.0,0.0,0.0
455215,300014702-353245971,254 rue de Vaugirard SO-NE,300014702,254 rue de Vaugirard,44.0,2021-08-09 10:00:00,2020-11-29,"48.83977,2.30198",Y2H20114504,48.839770,2.301980,3.806662,0.0,0.0,0.00,0.0,0.0,0.0


In [24]:
left_merged.info()

NameError: name 'left_merged' is not defined

## Feature extraction

To account for the temporal aspects of the data, we cannot input the `date` field directly into the model. Instead we extract the features on different time-scales from the `date` field, 

In [ ]:
def _encode_dates(X):
    X = X.copy()  # modify a copy of X
    # Encode the date information from the DateOfDeparture columns
    X.loc[:, "year"] = X["date"].dt.year
    X.loc[:, "month"] = X["date"].dt.month
    X.loc[:, "day"] = X["date"].dt.day
    X.loc[:, "weekday"] = X["date"].dt.weekday
    X.loc[:, "hour"] = X["date"].dt.hour

    # Finally we can drop the original columns from the dataframe
    return X.drop(columns=["date"])

In [ ]:
data["date"].head()

In [ ]:
_encode_dates(data[["date"]].head())

To use this function with scikit-learn estimators we wrap it with [FunctionTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.FunctionTransformer.html),

In [ ]:
from sklearn.preprocessing import FunctionTransformer

date_encoder = FunctionTransformer(_encode_dates, validate=False)
date_encoder.fit_transform(data[["date"]]).head()

Since it is unlikely that, for instance, that `hour` is linearly correlated with the target variable, we would need to additionally encode categorical features for linear models. This is classically done with [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), though other encoding strategies exist.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)

enc.fit_transform(_encode_dates(data[["date"]])[["hour"]].head())

## Linear model

Let's now construct our first linear model with [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html). We use a few helper functions defined in `problem.py` of the starting kit to load the public train and test data:

In [ ]:
merged_test = pd.merge(data_test, data_weather, how='left')
merged_test = merged_test.replace(np.nan, 0)

In [ ]:
target_name = 'log_bike_count'
X_train = merged_train.drop(columns=[target_name])
X_test = merged_test.drop(columns=[target_name])
y_train = merged_train.loc[:,target_name]
y_test = merged_test.loc[:,target_name]

In [ ]:
X_train.head(2)

and

Where `y` contains the `log_bike_count` variable. 

The test set is in the future as compared to the train set,

In [ ]:
print(
    f'Train: n_samples={X_train.shape[0]},  {X_train["date"].min()} to {X_train["date"].max()}'
)
print(
    f'Test: n_samples={X_test.shape[0]},  {X_test["date"].min()} to {X_test["date"].max()}'
)

In [ ]:
_encode_dates(X_train[["date"]]).columns.tolist()

We then evaluate this model with the RMSE metric,

In [ ]:
data_test = pd.read_parquet(Path("data") / "test.parquet")

In [ ]:
merged_test = pd.merge(data_test, data_weather, how='left')

In [ ]:
y_train = merged_train.loc[:,'log_bike_count']

In [ ]:
y_test = merged_test.loc[:,'log_bike_count']

## Linear for now

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline

date_encoder = FunctionTransformer(_encode_dates)
date_cols = _encode_dates(X_train[["date"]]).columns.tolist()

categorical_encoder = OneHotEncoder(handle_unknown="ignore")
categorical_cols = ["counter_name", "site_name"]
scaling_columns = ['pmer','ff','t','u','vv','pres']

preprocessor = ColumnTransformer(
    [
        ("date", OneHotEncoder(handle_unknown="ignore"), date_cols),
        ("cat", categorical_encoder, categorical_cols),
        ('standard-scaler', StandardScaler(), scaling_columns)
    ]
)

regressor = Ridge()

pipe = make_pipeline(date_encoder, preprocessor, regressor)
pipe.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error

print(
    f"Train set, RMSE={mean_squared_error(y_train, pipe.predict(X_train), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_test, pipe.predict(X_test), squared=False):.2f}"
)

## Let's visualize the predictions for one of the stations

In [ ]:
mask = (
    (X_test["counter_name"] == "Totem 73 boulevard de Sébastopol S-N")
    & (X_test["date"] > pd.to_datetime("2021/09/01"))
    & (X_test["date"] < pd.to_datetime("2021/09/08"))
)

df_viz = X_test.loc[mask].copy()
df_viz["bike_count"] = np.exp(y_test[mask.values]) - 1
df_viz["bike_count (predicted)"] = np.exp(pipe.predict(X_test[mask])) - 1

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

df_viz.plot(x="date", y="bike_count", ax=ax)
df_viz.plot(x="date", y="bike_count (predicted)", ax=ax, ls="--")
ax.set_title("Predictions with Ridge")
ax.set_ylabel("bike_count")

So we start to see the daily trend, and some of the week day differences are accounted for, however we still miss the details and the spikes in the evening are under-estimated.

A useful way to visualize the error is to plot `y_pred` as a function of `y_true`,

In [ ]:
pipe.predict(X_test).flatten()

In [ ]:
y_test.shape

In [ ]:
fig, ax = plt.subplots()

df_viz = pd.DataFrame({"y_true": y_test, "y_pred": pipe.predict(X_test).flatten()}).sample(
    10000, random_state=0
)

df_viz.plot.scatter(x="y_true", y="y_pred", s=8, alpha=0.1, ax=ax)

It is recommended to use cross-validation for hyper-parameter tuning with [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) or more reliable model evaluation with [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score). In this case, because we want the test data to always be in the future as compared to the train data, we can use [TimeSeriesSplit](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html),

<img src="https://i.stack.imgur.com/Q37Bn.png" />

The disadvantage, is that we can either have the training set size be different for each fold which is not ideal for hyper-parameter tuning (current figure), or have constant sized small training set which is also not ideal given the data periodicity. This explains that generally we will have worse cross-validation scores than test scores, 

In [ ]:
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

cv = TimeSeriesSplit(n_splits=6)

# When using a scorer in scikit-learn it always needs to be better when smaller, hence the minus sign.
scores = cross_val_score(
    pipe, X_train, y_train, cv=cv, scoring="neg_root_mean_squared_error"
)
print("RMSE: ", scores)
print(f"RMSE (all folds): {-scores.mean():.3} ± {(-scores).std():.3}")